# 2023 Zero Hunger Challenge - Master's in Business Analytics

# Introduction:

In 2023, the shocking reality persists: 800 million people still endure hunger. Despite remarkable technological advances, the scourge of climate change remains a formidable contributor to acute food insecurity. To address this, our challenge was to detect a specific crop in different geographic areas from An Giang province, Vietnam, with a particular focus on rice, which is one the most sought-after staples in the world.

# Top Actionable Recommendations:

1) Leverage Microsoft Planetary Computer Datasets for Comprehensive and Reliable Data:

Utilize the extensive and highly reliable Microsoft Planetary Computer datasets as a primary data source for this binary problem, they offer a wealth of satellite-based information, making them an invaluable resource for accurate land classification. 

2) Vegetation Indexes Calculation & Feature Engineering

Carefully engineer and select the most relevant features from your satellite data, such as different vegetation indices (NDVI & RVI) and its multiple ratios and combinations. 
These features will play a crucial role in the accuracy of your predictions.

3) Spatial Analysis and Proximity Features:

Consider that effective features for land classification do not always require complexity. Sometimes, the most impactful variables can be developed with ease, such as proximity to cities or proximity to existing rice lands.
Calculate distances measures to capture the influence of these factors on land classification. Geospatial libraries like GeoPandas can be used to perform spatial operations.

--------------------------

# Load in Dependencies

Multiple packages were loaded in to manage data import, data cleaning, analyses, visualizations, and machine learning models development.

In [2]:
# Supress Warnings
import warnings
warnings.filterwarnings('ignore')

# Visualization
import ipyleaflet
import matplotlib.pyplot as plt
from IPython.display import Image
import seaborn as sns
import plotly.express as px

# Import common Data Science and GIS tools
import numpy as np
import pandas as pd
import xarray as xr
import rioxarray as rio
import rasterio.features
from geopy.distance import geodesic

# Feature Engineering
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

# Machine Learning
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import NearestNeighbors
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import f1_score, accuracy_score,classification_report,confusion_matrix

# Planetary Computer Tools
import pystac
import pystac_client
import odc
import xrspatial.multispectral as ms
from pystac_client import Client
from pystac.extensions.eo import EOExtension as eo
from odc.stac import stac_load
import planetary_computer as pc
pc.settings.set_subscription_key('9db940db112543708a705a384288d31a')

# Others
import requests
import rich.table
from itertools import cycle
from tqdm import tqdm
tqdm.pandas()
from datetime import datetime

SyntaxError: invalid syntax (3803654446.py, line 1)

# Load in Datasets

Two datasets were provided by Challenge Administrators:
    
Dataset No.1: Contains 600 lands with the latitude and longitude data points and class of land values assigned. 

Dataset No.2: Contains 250 lands with the latitude and longitude data points only. Class of Land for these should be predicted.

In [ ]:
#Dataset No.1: Class of Land Data Provided
crop_presence_data = pd.read_csv("Crop_Location_Data_20221201.csv")
crop_presence_data.head()

In [ ]:
#Dataset No.2: Class of Land Data to be Predicted
crop_topredict_data = pd.read_csv("challenge_1_submission_template_correct_columns_fixed.csv")
crop_topredict_data.head()

To continue with analysis and new features development steps, an update in the headings of these Datasets is done for them to share same structure and concatenate them successfully into one dataframe. 

In [ ]:
#Updating the id and target columns to be Latitude and Longitude, and Class of Land respectively. Such as crop_presence_data.
crop_topredict_data.rename(columns={'id':'Latitude and Longitude','target':'Class of Land'},
                            inplace=True)
crop_topredict_data

In [3]:
#Concatenating both provided data and data to be predicted into one dataframe.
complete_crop_data= pd.concat([crop_presence_data, crop_topredict_data], ignore_index=True)
complete_crop_data

NameError: name 'crop_presence_data' is not defined

In [ ]:
complete_crop_data = complete_crop_data.rename(columns={'Latitude and Longitude': 'Latitude_and_Longitude', 'Class of Land': 'Class_of_Land'})

The main dataframe used in this notebook is called 'df' and now it contains 850 rows (300 are rice, 300 are non-rice, and 250 to be predicted) and 2 columns (Latitude and Longitude, and Class of Land)

In [ ]:
df = complete_crop_data

# Radar and Optical Data

Radar and optical data (Sentinel-1 and Sentinel-2 satellites data) was imported for the lands locations we have. For a fast performance on this main code, the lines developed to get data from the mentioned satellites were included into the Appendix. In this notebook, we are reading radar data and optical data from xlsx files that were outputted from the codes that can be found in Appendix.

In [ ]:
# Reading radar data to then calculate Radar Vegetation Index.
rvi_df = pd.read_excel('HULT_VV_VH_FullDataset.xlsx')

In [ ]:
# Converting datetime column into datetime format.
rvi_df['datetime'] = pd.to_datetime(rvi_df['datetime'])

In [ ]:
# Creating a new column 'Month_Year' from datetime column.
rvi_df['Month_Year'] = rvi_df['datetime'].dt.strftime('%b_%Y')

In [ ]:
# Dropping columns that are not necessary.
columns_to_drop = ['id_Full_Dataset','datetime','id_Crop_Data','fl_Train','fl_Test']
rvi_df.drop(columns_to_drop, axis=1,inplace=True)

In [ ]:
# Calculating mean VV and VH values grouping by Month_Year, Coordinates and Class of Land.
mean_values = rvi_df.groupby(['Month_Year', 'Latitude_and_Longitude','Class_of_Land'])[['VV', 'VH']].mean().reset_index()

In [ ]:
# Calculating RVI for each row
def calculate_rvi(row):
    dop = row['VV'] / (row['VV'] + row['VH'])
    m = 1 - dop
    rvi = np.sqrt(dop) * ((4 * row['VH']) / (row['VV'] + row['VH']))
    return rvi

mean_values['RVI'] = mean_values.apply(calculate_rvi, axis=1)
# Now, 'mean_values' contains the 'RVI' values for each 'Month_Year' and 'Latitude_and_Longitude'

In [ ]:
# Pivotting the DataFrame
pivoted_df = mean_values.pivot(index='Latitude_and_Longitude', columns='Month_Year', values='RVI')

# Checking if 'Dec_2022' exists in the columns, and if not, insert it with NaN values
if 'Dec_2022' not in pivoted_df.columns:
    pivoted_df['Dec_2022'] = pd.NA
    
# Creating a custom order for 'Month_Year' from Jan 2021 to Nov 2022
custom_order = [f"{month}_{year}" for year in range(2021, 2023) for month in ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']]

# Reordering columns based on the custom order
pivoted_df = pivoted_df[custom_order]

# Resetting the index to make 'Latitude_and_Longitude' a regular column
pivoted_df.reset_index(inplace=True)

In [ ]:
# Dropping Dec_2022 because there are no values for that Month_Year but we needed it for custom order of columns
pivoted_df.drop(['Dec_2022'], axis=1, inplace=True)

In [ ]:
# Merging the 'Class_of_Land' column from 'df' with 'pivoted_df' based on 'Latitude_and_Longitude'
merged_df = pd.merge(pivoted_df, df[['Latitude_and_Longitude', 'Class_of_Land']], on='Latitude_and_Longitude', how='left')

# Now, 'merged_df' contains 'Month_Year' columns, 'Latitude_and_Longitude', and 'Class_of_Land' columns

In [ ]:
rvi_df=merged_df

Now, 'rvi_df' dataframe contains multiple columns that show RVI value per Month_Year for each Land.

In [ ]:
rvi_df

In [ ]:
# Reading optical data to then calculate Normalized Difference Vegetation Index and s2_fl_vegetation.
ndvi = pd.read_csv("Sentinel2_12BANDS.csv")
ndvi.head()

Following code sums s2_fl_vegetation column per Land.

In [ ]:
# Creating a new dataframe that only include these 3 columns.
df_vegetation = ndvi[['Latitude_and_Longitude','datetime','s2_fl_Vegetation']]
df_vegetation

In [ ]:
# Converting 'datetime' column into datetime format.
df_vegetation['datetime'] = pd.to_datetime(df_vegetation['datetime'])

In [ ]:
df_vegetation['Month_Year'] = df_vegetation['datetime'].dt.strftime('%b_%Y')
df_vegetation

In [ ]:
# Defining the list of months to include
included_months = ['Nov_2021', 'Dec_2021', 'Jan_2022', 'Feb_2022', 'Mar_2022', 'Apr_2022','Jun_2022', 'Jul_2022', 'Aug_2022']

# Filtering the DataFrame to include only rows with 'Month_Year' values in the specified list
df_vegetation = df_vegetation[df_vegetation['Month_Year'].isin(included_months)]

In [ ]:
grouped = df_vegetation.groupby('Latitude_and_Longitude')['s2_fl_Vegetation'].sum()

# Creating a new DataFrame with the 'Latitude_and_Longitude' and 'Sum_Vegetation'
df_vegetation = grouped.reset_index()

df_vegetation

Now overall s2_fl_Vegetation was summarized by Land.
We proceed to calculate NDVI mean per Month_Year for each Land.

In [ ]:
ndvi_df = ndvi[['Class_of_Land','Latitude_and_Longitude','datetime','s2_NVDI']]

In [ ]:
ndvi_df['datetime'] = pd.to_datetime(ndvi_df['datetime'])

In [ ]:
ndvi_df['Month_Year'] = ndvi_df['datetime'].dt.strftime('%b_%Y')+'_NDVI'

In [ ]:
ndvimean_values = ndvi_df.groupby(['Month_Year', 'Latitude_and_Longitude','Class_of_Land'])[['s2_NVDI']].mean().reset_index()

In [ ]:
# Pivotting the DataFrame
ndvi_df = ndvimean_values.pivot(index='Latitude_and_Longitude', columns='Month_Year', values='s2_NVDI')

In [ ]:
# Checking if 'Dec_2022' and 'May_2022' exist in the columns, and if not, insert it with NaN values
if 'Dec_2022_NDVI' not in ndvi_df.columns:
    ndvi_df['Dec_2022_NDVI'] = pd.NA
    
if 'May_2022_NDVI' not in ndvi_df.columns:
    ndvi_df['May_2022_NDVI'] = pd.NA

# Adding 'NDVI' to the custom order for 'Month_Year'
custom_order_ndvi = [f"{month}_{year}_NDVI" for year in range(2021, 2023) for month in ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']]

# Reordering columns based on the custom order
ndvi_df = ndvi_df[custom_order_ndvi]

# Resetting the index to make 'Latitude_and_Longitude' a regular column
ndvi_df.reset_index(inplace=True)

In [ ]:
ndvi_df

Now we have 3 dataframes: ndvi_df, df_vegetation, rvi_df.

These three are merged using Latitude and Longitude into one final dataframe called 'df'.

In [ ]:
df = pd.merge(ndvi_df,rvi_df,on='Latitude_and_Longitude',how='inner')

In [ ]:
df = pd.merge(df,df_vegetation,on='Latitude_and_Longitude',how='inner')

In [ ]:
df

In [ ]:
# Splitting Latitude and Longitude columns into separate columns
df[['Latitude', 'Longitude']] = df['Latitude_and_Longitude'].str.replace('(','').str.replace(')','').str.replace(' ','').str.split(',', expand=True)
df

# Closeness to Cities

A new feature not related to Vegetation indexes is added: Closeness to city. 
To calculate this, we used worldcities dataset from https://simplemaps.com/data/world-cities. 
find_closest_city function calculates the closest distance in kilometers and closest city each Land has based on coordinates (lat,long).

In [ ]:
# Calculating distance to cities
cities_df = pd.read_csv('worldcities.csv')
cities_df

In [ ]:
def find_closest_city(row): #We define the function as 'find_closest_city' that just takes 'row' as single argument
    min_distance = float('inf') #Initializes the variable with positive infinity to then store the min distance found 
    closest_city = "" #Initializes this variable as empty string, will store the name of the closest city afterwards

    for index, city_row in tqdm(cities_df.iterrows(), total=len(cities_df), desc="Calculating"): #starts a loop that iterates over the rows of 'cities_df' dataframe. tqdm was added to see progress of application. City_eow is the data in that row, and 'index' is index of current row.
        city_coords = (city_row['lat'], city_row['lng']) #extracts the lat and long from current city_row
        distance = geodesic((row['Latitude'], row['Longitude']), city_coords).kilometers #calculates distance between greographical point and city. The result is stored in kilometers in 'distance' variable.

        if distance < min_distance: #checks if calculated 'distance' is less than the current 'min_distance'. If it is, this means current city is closer to the input point.
            min_distance = distance #if condition is met, this variable is updated with the new, smaller 'distance'
            closest_city = city_row['city_ascii'] #if condition is met, this variable is updated with the name of the city from current 'city_row'

    return pd.Series([min_distance, closest_city]) #after loop is completed, function returns a series with two elements: 'min_distance' and 'closest_city'

In [ ]:
# Applying the function to each row in 'df' and creating new closest distance and closest city columns
df[['Closest_Distance', 'Closest_City']] = df.apply(find_closest_city, axis=1)
df

# Feature Engineering

Multiple new features based on statistical concepts were developed to help predict Class of Land target.

In [ ]:
# Defining RVI feature columns
rvi_feature_columns = ['Nov_2021', 'Dec_2021', 'Jan_2022', 'Feb_2022', 'Mar_2022', 'Apr_2022', 'May_2022', 'Jun_2022', 'Jul_2022', 'Aug_2022']

In [ ]:
# Defining RVI feature columns
ndvi_feature_columns = ['Nov_2021_NDVI','Dec_2021_NDVI','Jan_2022_NDVI','Feb_2022_NDVI','Mar_2022_NDVI','Apr_2022_NDVI','May_2022_NDVI','Jun_2022_NDVI','Jul_2022_NDVI','Aug_2022_NDVI']

In [ ]:
# Aggregating RVI values for each month
df['mean_RVI'] = df[rvi_feature_columns].mean(axis=1)
df['std_RVI'] = df[rvi_feature_columns].std(axis=1)
df['min_RVI'] = df[rvi_feature_columns].min(axis=1)
df['max_RVI'] = df[rvi_feature_columns].max(axis=1)
df['DifferenceMinMaxRVI'] = df['max_RVI'] - df['min_RVI']
df['RVI_Variability_to_Mean_RVI'] = df['std_RVI'] / df['mean_RVI']

In [ ]:
# Aggregating NDVI values for each month
df['mean_NDVI'] = df[ndvi_feature_columns].mean(axis=1)
df['std_NDVI'] = df[ndvi_feature_columns].std(axis=1)
df['min_NDVI'] = df[ndvi_feature_columns].min(axis=1)
df['max_NDVI'] = df[ndvi_feature_columns].max(axis=1)
df['DifferenceMinMaxNDVI'] = df['max_NDVI'] - df['min_NDVI']
df['NDVI_Variability_to_Mean_NDVI'] = df['std_NDVI'] / df['mean_NDVI']

We noticed that NDVI had its greatest fluctuation between Nov_2021 and Mar_2022, reason why fluctuation value between these two months was calculated as a new column.

In [ ]:
df['NDVI_Fluctuation_Mar22Nov21'] = df['Mar_2022_NDVI'] - df['Nov_2021_NDVI']

In [ ]:
# Deleting columns we do not need. The challenge is focused on this period of time: November 2021 until August 2022
df.drop(['Jan_2021_NDVI', 'Feb_2021_NDVI','Mar_2021_NDVI','Apr_2021_NDVI','May_2021_NDVI','Jun_2021_NDVI','Jul_2021_NDVI','Aug_2021_NDVI','Sep_2021_NDVI','Oct_2021_NDVI','Sep_2022_NDVI','Oct_2022_NDVI','Nov_2022_NDVI','Dec_2022_NDVI','Jan_2021','Feb_2021','Mar_2021','Apr_2021','May_2021','Jun_2021','Jul_2021','Aug_2021','Sep_2021','Oct_2021','Sep_2022','Oct_2022','Nov_2022'], axis=1, inplace=True)

The final version of the dataset used to generate predictions is saved as CSV.

In [ ]:
df.to_csv('FinalDataset_submission.csv')

# Correlations

Next steps are related to prepare data for visualizing and developing future machine learning algorithms:
- Class of land values were replaced to be numerical
    
- Latitude and Longitude values were converted to float data type
   
- 'df' dataframe was splitted into 2: df_known (where class of land is not null) & df_unknown (where class of land is null)

In [ ]:
df['Class_of_Land'].replace({'Rice': 1, 'Non Rice': 0}, inplace=True)

In [ ]:
# Converting Latitude and Longitude columns to float data type
df['Latitude'] = df['Latitude'].astype(float)
df['Longitude'] = df['Longitude'].astype(float)

In [ ]:
df_known = df[df['Class_of_Land'].notna()]  # Contains 600 rows with known Land_Class
df_unknown = df[df['Class_of_Land'].isna()]  # Contains 250 rows with unknown Land_Class

df_known.corr()

Top 10 highest correlations were calculated:

In [ ]:
correlations = df_known.corr()['Class_of_Land'].abs().sort_values(ascending=False)
top_correlations = correlations[1:11]  # Excluding the target variable itself

print("Top 10 highest correlations:")
print(top_correlations)

# Data Visualizations

Before moving on with machine learning steps, we created some plots for better data analysis.

In [ ]:
# Separating the rows with Class_of_Land 1 (Rice) and 0 (Non Rice)
rice_df = df_known[df_known['Class_of_Land'] == 1]
non_rice_df = df_known[df_known['Class_of_Land'] == 0]

# Calculating the mean for the specified columns
columns_to_average = ['Nov_2021', 'Dec_2021', 'Jan_2022', 'Feb_2022', 'Mar_2022', 'Apr_2022', 'May_2022', 'Jun_2022', 'Jul_2022', 'Aug_2022']

rice_mean = rice_df[columns_to_average].mean()
non_rice_mean = non_rice_df[columns_to_average].mean()

# Creating a new DataFrame with the means of the two classes
plotresult_df = pd.DataFrame({'Class_of_Land': [1, 0],  # 1 for Rice, 0 for Non Rice
                          'Nov_2021': [rice_mean['Nov_2021'], non_rice_mean['Nov_2021']],
                          'Dec_2021': [rice_mean['Dec_2021'], non_rice_mean['Dec_2021']],
                          'Jan_2022': [rice_mean['Jan_2022'], non_rice_mean['Jan_2022']],
                          'Feb_2022': [rice_mean['Feb_2022'], non_rice_mean['Feb_2022']],
                          'Mar_2022': [rice_mean['Mar_2022'], non_rice_mean['Mar_2022']],
                          'Apr_2022': [rice_mean['Apr_2022'], non_rice_mean['Apr_2022']],
                          'May_2022': [rice_mean['May_2022'], non_rice_mean['May_2022']],
                          'Jun_2022': [rice_mean['Jun_2022'], non_rice_mean['Jun_2022']],
                          'Jul_2022': [rice_mean['Jul_2022'], non_rice_mean['Jul_2022']],
                          'Aug_2022': [rice_mean['Aug_2022'], non_rice_mean['Aug_2022']]
                         })

# The result_df now contains the means of the specified columns for Class_of_Land 1 (Rice) and Class_of_Land 0 (Non Rice).

In [ ]:
# Transposing the DataFrame for easier plotting
sns.set_palette("Set1")
result_df_transposed = plotresult_df.transpose()

# Setting the column names for the legend
result_df_transposed.columns = ['Rice', 'Non Rice']

# Excluding the "Class of Land" column
result_df_transposed = result_df_transposed[1:]  # Excludes the first row (Class of Land)

# Creating the line chart
sns.set_style("whitegrid")
plt.figure(figsize=(12, 6))

# Plotting the line chart
sns.lineplot(data=result_df_transposed,palette=["blue","red"])

# Adding oval markers at data points
sns.scatterplot(data=result_df_transposed, markers=["o", "o"], legend=False, s=100,palette=["blue","red"])

plt.title("RVI Fluctuation Over Time by Class of Land", fontweight='bold')
plt.xlabel("Time Period", fontweight='bold')
plt.ylabel("Mean RVI Value", fontweight='bold')
plt.legend(title="Class of Land", loc="center left", bbox_to_anchor=(1, 0.5))
plt.xticks(rotation=45)
plt.show()


This plot was created by calculating the MEAN RVI value per month for each Class of Land (we have 300 rows for each class of land), and then plotting the data points. 
This lineplot shows the fluctuation Rice lands have on their RVI values over periods of time. Whereas non rice lands are mostly stable month over month.
Rice Lands have high peaks on RVI values on certain months, denoting the growth progress on their crop 'structure'.

In [ ]:
# Creating a histogram with 'Class_of_Land' as hue
plt.figure(figsize=(12, 6))
sns.set_style("whitegrid")
sns.histplot(data=df_known, x='RVI_Variability_to_Mean_RVI', hue='Class_of_Land',palette='Set1')

# Adding labels and title
plt.xlabel("RVI Variability to Mean RVI",fontweight='bold')
plt.ylabel("Frequency",fontweight='bold')
plt.title("RVI Variability to Mean RVI by Class of Land",fontweight='bold')

# Showing the legend
plt.legend(title="Class of Land", labels=['Rice', 'Non Rice'])

plt.show()

This histogram shows how variable the RVI values are relative to their average. A value close to 0 indicates that the RVI values are relatively stable. 

In [ ]:
# Calculating the correlation between 'Class_of_Land' and 'Closeness to Cities'
correlation = df_known['Class_of_Land'].corr(df_known['Closest_Distance'])

# Creating a scatter plot to visualize the relationship with the specified HEX color
plt.figure(figsize=(8, 6))
sns.scatterplot(x='Closest_Distance', y='Class_of_Land', data=df_known, color='#2C84E4')  # Set the color to #2C84E4

plt.title(f'Correlation: {correlation:.2f}', fontsize=14, fontweight='bold')  # Increase title font size
plt.xlabel('Closeness to Cities in KMs', fontsize=12, fontweight='bold')  # Increase X-axis label font size
plt.ylabel('Class of Land', fontsize=12, fontweight='bold')  # Increase Y-axis label font size
plt.yticks([0, 1], labels=['Non Rice', 'Rice'])  # Label the y-axis ticks

# Customizing the gridlines
sns.set_style("whitegrid")
plt.grid(True, linestyle='--', alpha=0.6)

plt.show()


It is clear how as km of distance to cities increase, there is a strong tendency for the "Class of Land" to most probably be Rice. So we can think that rice lands are not likely close to Cities. 

Non Rice Lands are typically found on a radar not beyond ~ 13 kilometers from a city, whereas 'Rice Lands' are often further located from cities, between 13 and 25 kilometers away.

In [ ]:
# Calculating the correlation between 'Class_of_Land' and 'RVI Standard Deviation'
correlation = df_known['Class_of_Land'].corr(df_known['std_RVI'])

# Creating a scatter plot to visualize the relationship
plt.figure(figsize=(8, 6))
sns.scatterplot(x='std_RVI', y='Class_of_Land', data=df_known, color='#2C84E4') 

plt.title(f'Correlation: {correlation:.2f}', fontsize=14, fontweight='bold') 
plt.xlabel('RVI Standard Deviation', fontsize=12, fontweight='bold') 
plt.ylabel('Class of Land', fontsize=12, fontweight='bold')  
plt.yticks([0, 1], labels=['Non Rice', 'Rice']) 

# Customizing the gridlines
sns.set_style("whitegrid")
plt.grid(True, linestyle='--', alpha=0.6)

plt.show()


This plot indicates how as the dispersion in RVI set of values increases, there is a strong tendency for the "Class of Land" to most probably by Rice. So we can think that rice lands have greater diversity on RVI values. The difference between their minimum RVI and maximum RVI values is usually big.

Next lines convert data changes to plot a line chart for NDVI fluctuation over periods of time.

In [ ]:
# Separating the rows with Class_of_Land 1 (Rice) and 0 (Non Rice)
rice_df = df_known[df_known['Class_of_Land'] == 1]
non_rice_df = df_known[df_known['Class_of_Land'] == 0]

# Calculating the mean for the specified columns
columns_to_average = ['Nov_2021_NDVI', 'Dec_2021_NDVI', 'Jan_2022_NDVI', 'Feb_2022_NDVI', 'Mar_2022_NDVI', 'Apr_2022_NDVI', 'May_2022_NDVI', 'Jun_2022_NDVI', 'Jul_2022_NDVI', 'Aug_2022_NDVI']

rice_mean = rice_df[columns_to_average].mean()
non_rice_mean = non_rice_df[columns_to_average].mean()

# Creating a new DataFrame with the means of the two classes
plotresult_df = pd.DataFrame({'Class_of_Land': [1, 0],  # 1 for Rice, 0 for Non Rice
                          'Nov_2021_NDVI': [rice_mean['Nov_2021_NDVI'], non_rice_mean['Nov_2021_NDVI']],
                          'Dec_2021_NDVI': [rice_mean['Dec_2021_NDVI'], non_rice_mean['Dec_2021_NDVI']],
                          'Jan_2022_NDVI': [rice_mean['Jan_2022_NDVI'], non_rice_mean['Jan_2022_NDVI']],
                          'Feb_2022_NDVI': [rice_mean['Feb_2022_NDVI'], non_rice_mean['Feb_2022_NDVI']],
                          'Mar_2022_NDVI': [rice_mean['Mar_2022_NDVI'], non_rice_mean['Mar_2022_NDVI']],
                          'Apr_2022_NDVI': [rice_mean['Apr_2022_NDVI'], non_rice_mean['Apr_2022_NDVI']],
                          'May_2022_NDVI': [rice_mean['May_2022_NDVI'], non_rice_mean['May_2022_NDVI']],
                          'Jun_2022_NDVI': [rice_mean['Jun_2022_NDVI'], non_rice_mean['Jun_2022_NDVI']],
                          'Jul_2022_NDVI': [rice_mean['Jul_2022_NDVI'], non_rice_mean['Jul_2022_NDVI']],
                          'Aug_2022_NDVI': [rice_mean['Aug_2022_NDVI'], non_rice_mean['Aug_2022_NDVI']]
                         })

# The result_df now contains the means of the specified columns for Class_of_Land 1 (Rice) and Class_of_Land 0 (Non Rice).

In [ ]:
# Transposing the DataFrame for easier plotting
result_df_transposed = plotresult_df.transpose()

# Setting the column names for the legend
result_df_transposed.columns = ['Rice', 'Non Rice']

# Excluding the "Class of Land" column
result_df_transposed = result_df_transposed[1:]  # Excludes the first row (Class of Land)

# Creating the line chart
sns.set_style("whitegrid")
plt.figure(figsize=(12, 6))

# Plotting the line chart
sns.lineplot(data=result_df_transposed,palette=["blue","red"])

# Adding oval markers at data points
sns.scatterplot(data=result_df_transposed, markers=["o", "o"], legend=False, s=100,palette=["blue","red"])

plt.title("NDVI Fluctuation Over Time by Class of Land", fontweight='bold')
plt.xlabel("Time Period", fontweight='bold')
plt.ylabel("Mean NDVI Value", fontweight='bold')
plt.legend(title="Class of Land", loc="center left", bbox_to_anchor=(1, 0.5))
plt.xticks(rotation=45)
plt.show()

The plot shows how NDVI values from Rice lands usually are more diverse than Non rice NDVI values. There is a big fluctuation from Nov_2021 until Mar_2021 (peak of greeness).
Then, it is visible how after june the greenness visibility starts to drastically decrease mainly due to harvesting.

In [ ]:
# Calculating the correlation between 'Class_of_Land' and 'NDVI_Fluctuation_Mar22Nov21'
correlation = df_known['Class_of_Land'].corr(df_known['NDVI_Fluctuation_Mar22Nov21'])

# Creating a scatter plot to visualize the relationship with the specified HEX color
plt.figure(figsize=(8, 6))
sns.scatterplot(x='NDVI_Fluctuation_Mar22Nov21', y='Class_of_Land', data=df_known, color='#2C84E4') 

plt.title(f'Correlation: {correlation:.2f}', fontsize=14, fontweight='bold')  
plt.xlabel('NDVI Fluctuation Between Nov21 and Mar22', fontsize=12, fontweight='bold') 
plt.ylabel('Class of Land', fontsize=12, fontweight='bold')  
plt.yticks([0, 1], labels=['Non Rice', 'Rice'])

# Customizing the gridlines
sns.set_style("whitegrid")
plt.grid(True, linestyle='--', alpha=0.6)

plt.show()


There is a correlation between fluctuation of NDVI values between Nov21 and Mar22. Rice lands tend to have greater NDVI fluctuation than non rice lands.

# Machine Learning Algorithm: Set Up and Training

All features column are defined, as well as target column (Class of Land). 

The dataframe that contains the 600 rows with class of lands values is splitted between test and train. 20% of the data would be used to test.

The data is also standardized to ensure the model performs optimally.

Random Forest Classifier is the algorithm chosen to solve this binary problem. 

The classification report, accuracy value, confusion matrix, and F1 score determined this model is a great solution.  

In [ ]:
feature_columns = ['Nov_2021_NDVI', 'Dec_2021_NDVI', 'Jan_2022_NDVI', 'Feb_2022_NDVI','Mar_2022_NDVI', 'Apr_2022_NDVI',
                   'Jun_2022_NDVI','Jul_2022_NDVI', 'Aug_2022_NDVI','Nov_2021', 'Dec_2021', 'Jan_2022',
                   'Feb_2022', 'Mar_2022','Apr_2022', 'May_2022', 'Jun_2022', 'Jul_2022', 'Aug_2022','s2_fl_Vegetation',
                   'Closest_Distance','mean_RVI', 'std_RVI','min_RVI', 'max_RVI', 'DifferenceMinMaxRVI',
                   'RVI_Variability_to_Mean_RVI', 'mean_NDVI', 'std_NDVI', 'max_NDVI', 'DifferenceMinMaxNDVI',
                   'NDVI_Variability_to_Mean_NDVI','NDVI_Fluctuation_Mar22Nov21']

In [ ]:
target_column = 'Class_of_Land'

In [ ]:
# Splitting the data into training and testing sets
X = df_known[feature_columns]
y = df_known[target_column]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Standardizing the feature data
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Creating and train a machine learning model (Random Forest Classifier being used)
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

# Making predictions on the test set
y_pred = model.predict(X_test)

# Evaluating the model
accuracy = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred)

print(f'Accuracy: {accuracy}')
print(f'Classification Report:\n{report}')

In [ ]:
# Calculate the confusion matrix
confusion = confusion_matrix(y_test, y_pred)

# Print the confusion matrix
print("Confusion Matrix:")
print(confusion)

In [ ]:
# Calculating the F1 score
f1 = f1_score(y_test, y_pred)

print(f'F1 Score: {f1}')

# Machine Learning Model: Calculating Predictions

In [ ]:
# Standardizing the feature data 
X_unknown = scaler.transform(df_unknown[feature_columns])

# Making predictions on the unknown data using the trained model
y_unknown_pred = model.predict(X_unknown)

# Adding the predicted 'Class_of_Land' values to df_unknown DataFrame
df_unknown['Predicted_Class_of_Land'] = y_unknown_pred

In [ ]:
# Creating a new DataFrame with 'Original_Latitude', 'Original_Longitude', and 'Predicted_Class_of_Land'
df_result = df_unknown[['Latitude_and_Longitude','Predicted_Class_of_Land']]

In [ ]:
# Renaming 'Predicted_Class_of_Land' to 'target' and map the values 0 and 1
df_result = df_result.rename(columns={'Predicted_Class_of_Land': 'target'})

# Mapping the target values: 0 = Non Rice, 1 = Rice
df_result['target'] = df_result['target'].map({0: 'Non Rice', 1: 'Rice'})

In [ ]:
# Renaming 'Predicted_Class_of_Land' to 'target' and map the values 0 and 1
df_result = df_result.rename(columns={'Latitude_and_Longitude': 'id'})

Prediction is done now on the df_unknown dataframe (dataset that contains 250 lands with class of land to be predicted).
And some headings renaming are done.

# Closeness to Rice Lands

Rice Lands usually tend to be close to each other because they share similar surface characteristics. Next lines of code perform a spatial analysis to adjust land classification predictions based on the majority class of nearby land points (n_neighbors=20).

In [ ]:
# Resetting the index of the DataFrame to ensure it aligns correctly
df_result = df_result.reset_index(drop=True)

# Extracting coordinates from the 'Latitude_and_Longitude' column and convert them to float
df_result['Latitude'] = df_result['id'].apply(lambda x: float(x.split(',')[0].strip('()')))
df_result['Longitude'] = df_result['id'].apply(lambda x: float(x.split(',')[1].strip('()')))

# Specifying the number of nearest neighbors to consider
n_neighbors = 20  

# Creating a NearestNeighbors model
coordinates = df_result[['Latitude', 'Longitude']].values
nbrs = NearestNeighbors(n_neighbors=n_neighbors, algorithm='ball_tree').fit(coordinates)

# Finding the distances and indices of the nearest neighbors for each land
distances, indices = nbrs.kneighbors(coordinates)

# Function to adjust predictions based on neighbors
def adjust_prediction(row):
    neighbor_indices = indices[row.name]
    neighbor_classes = df_result['target'].iloc[neighbor_indices]
    
    # Counting the number of Rice and Non Rice neighbors
    rice_count = np.sum(neighbor_classes == 'Rice')
    non_rice_count = np.sum(neighbor_classes == 'Non Rice')
    
    # If the majority of neighbors are Rice, adjusting the prediction to Rice
    if rice_count >= non_rice_count:
        return 'Rice'
    else:
        return 'Non Rice'

# Appling the adjustment function to your predictions in 'df_result'
df_result['Adjusted_Class_of_Land'] = df_result.apply(adjust_prediction, axis=1)

# Dropping the temporary 'Latitude' and 'Longitude' columns if desired
df_result.drop(['Latitude', 'Longitude'], axis=1, inplace=True)

Final model results is saved as CSV file after some formatting changes.

In [ ]:
# Dropping the 'target' column
df_result.drop(columns=['target'], inplace=True)

# Renaming the 'Adjusted_Class_of_Land' column to 'target'
df_result.rename(columns={'Adjusted_Class_of_Land': 'target'}, inplace=True)

# Selecting only the 'id' and 'target' columns
df_result = df_result[['id', 'target']]

# Saving the DataFrame to a CSV file
df_result.to_csv('challenge_1_submission_group3.csv', index=False)

# Visualizing Results in Map

In [ ]:
# Splitting Latitude and Longitude columns into separate columns
df_result[['Latitude', 'Longitude']] = df_result['id'].str.replace('(','', regex=True).str.replace(')','', regex=True).str.replace(' ','', regex=True).str.split(',', expand=True)

# plotting map using plotly
fig = px.scatter_mapbox(df_result, 
                        lat='Latitude',
                        lon='Longitude',
                        color='target',
                        center={'lat':10.4391,
                                'lon':105.3},
                        zoom=8)

fig.update_layout(mapbox_style='open-street-map')

fig.show()

# Conclusion:

In the pursuit of decreasing world hunger, our project embarked on an impactful challenge to predict the class of lands in An Giang province, Vietnam. With Python as our trusted ally, we set out to distinguish between Rice and Non Rice lands, leveraging the power of satellite data and taking advantage of high valuable information available through Microsoft Planetary Computer. 

The Sentinel-1 and Sentinel-2 satellites provided the bedrock of our analysis. From Sentinel-1, we extracted VV and VH values, enabling us to construct the Radar Vegetation Index (RVI). This index unveiled the structural characteristics of crops. Meanwhile, the Normalized Difference Vegetation Index (NDVI) derived from Sentinel-2 data, offered insights into the dynamic "greenness" of the land, an invaluable indicator of the crop's growth stage, from planting to harvest stages.

In our endeavor to enhance the precision of our predictions, we incorporated an additional dimension—proximity to cities. This information revealed a distinctive trend: rice lands tended to be situated farther from urban centers.

Last but not least, one other pivotal aspect of our project resided in the field of spatial analysis. We leveraged the K Nearest Neighbors algorithm as last step, taking into account the collective wisdom of the 20 nearest neighboring lands. When the majority of these neighbors declared themselves as 'Rice' lands, the decision was adjusting the prediction to match their nearby lands, recognizing the spatial interdependencies in our data.

As we wrap up this endeavor, we firmly believe our predictions will act as guidance in the fight against hunger. By working toghether employing cutting-edge technology, harnessing satellite data, considering geographic relationships, and incorporating spatial insights, we have taken a meaningful step towards addressing one of the world's most pressing challenges with an F1 score of 1.00. Our project stands as a testament to the boundless potential of data-driven approaches in the quest for a hunger-free world.

# Bibliography:

https://simplemaps.com/data/world-cities | Site used to download cities dataset to then calculate closeness to cities.

https://custom-scripts.sentinel-hub.com/sentinel-2/cby_cloud_detection/ | Site used to get more information around cloud filtering applied on Appendix code.

https://explorer.digitalearth.africa/products/s2_l2a | Site used to get more information around bands descriptions to calculate NDVI and other features.

# Appendix:

- Attached to submitted files
- Sentinel-1: APPENDIX_Planetary_Computer_Info_Group3_Sentinel1_VV_VH _202310
- Sentinel-2: APPENDIX_Planetary_Computer_Info_Group3_Sentinel2_12Bands_202310
- Used to get data needed from Microsoft Planetary Computer to calculate RVI and NDVI values.